In [ ]:
outputfolder="SW"
folder(outputfolder)

for windows_size in [2,6,9]:
    for j in files_add:
        print(j[5:])
        df=pd.read_csv(j)#,usecols=n)

        df["sport"]=df["UDP_sport"]+df["TCP_sport"]
        df["dport"]=df["UDP_dport"]+df["TCP_dport"]

        del df["UDP_sport"]
        del df["TCP_sport"]
        del df["UDP_dport"]
        del df["TCP_dport"]

        df.to_csv("temp.csv")


        df=pd.read_csv("temp.csv")
        #df["ID"] = df["Ether_src"] +"@" +df["IP_src"] +"====>"+df["Ether_dst"]+ "@"+ df["IP_dst"]
        df["ID"] = df["WS_src"]+"=>"+df["WS_dst"]
        #del df["Ether_src"] 
        #del df["IP_src"] 
        #del df["Ether_dst"] 
        #del df["IP_dst"]
        del df["WS_src"] 
        del df["WS_dst"]

        cols=[ 
     'TCP_SYN',
     'TCP_ACK', 'ID', 'pck_size_diff', 'pck_size_mean',
               'pck_size_std',  'ts_diff', 'ts_mean', 'ts_std',
                'TCP_window_diff', 'TCP_window_mean', 'TCP_window_std'
                ,"payload_bytes_diff","payload_bytes_mean","payload_bytes_std",
            
'pck_size_mean_EW','pck_size_std_EW',
'ts_mean_EW', 'ts_std_EW',
'TCP_window_mean_EW','TCP_window_std_EW',
'payload_bytes_mean_EW','payload_bytes_std_EW',
'ts_sum_of_EW', 'pck_size_sum_of_EW', 
'payload_bytes_sum_of_EW','entropy_sum_of_EW','TCP_window_sum_of_EW',"Label"]
        df[df.columns[-1]]=df[df.columns[-1]].astype(str)
        IDS=sorted(list(df[df.columns[-1]].unique()))
        func_sw=[mean,std] #functions
        func_ew=[diff,mean_EW,std_EW,sum_of_EW] #functions   
        func_name_sw=["mean","std"]
        func_name_ew=["diff","mean_WE","std_WE","sum_of_EW"]
        
        f_list=["pck_size","ts",'TCP_window']#, "dport","sport"     'TCP_SYN', 'TCP_ACK'    'TCP_FIN', 'TCP_SYN', 'TCP_RST', 'TCP_PSH', 'TCP_ACK', 'TCP_URG', 'TCP_ECE', 'TCP_CWR',  
        fark=[#'pck_size_diff',
         'pck_size_mean',
         'ts_mean',
         'ts_std',
         'TCP_window_mean',
         'TCP_window_std'
         ]
        WS=windows_size
        flag=1
        for i in tqdm(IDS):
            smaller = df[df["ID"]==i]
            smaller=smaller.reset_index()
            del smaller["index"]
            smaller["ts"]=smaller["ts"].values-smaller["ts"].values[0]
            for ii in f_list:
                for jjj,iii in enumerate (func_ew):
                    name=str(ii)+"_"+str(func_name_ew[jjj])
                    smaller[name]=iii(smaller[ii])
            if len(smaller)>=WS:
                for ii in f_list:
                    for jjj,iii in enumerate (func_sw):
                        name=str(ii)+"_"+str(func_name_sw[jjj])
                        smaller[name]=iii(smaller[ii],WS)
            else:
                for ii in fark:
                        smaller[ii]=0
            smaller['TCP_SYN_sum'  ]=flagsum (smaller['TCP_SYN'  ].values)
            smaller['TCP_ACK_sum'  ]=flagsum (smaller['TCP_ACK'  ].values)
 
            smaller["TCP_SYN_ratio"]=smaller["TCP_SYN"]/(smaller["TCP_SYN_sum"]+10e-20)
            smaller["TCP_ACK_ratio"]=smaller["TCP_ACK"]/(smaller["TCP_ACK_sum"]+10e-20)
            col_list= ["TCP_SYN_sum","TCP_ACK_sum"]
            smaller['sum'] = smaller[col_list].sum(axis=1)
            smaller["TCP_ACK_SR"]=smaller["TCP_ACK"]/(smaller["sum"]+10e-20)
            
            smaller["TCP_ACK_R"]=smaller["TCP_ACK_sum"]/(smaller["sum"]+10e-20)
    
            if flag:
                smaller.to_csv(j[5:],header=True,index=False)
                flag=0
            else:
                smaller.to_csv(j[5:],header=False,index=False,mode="a")
        df=pd.read_csv(j[5:])
        features=df.columns
        count=[]
        bos=[]
        df=df.sort_values(by='Unnamed: 0')

        y=df["Label"]
        #for i in bos:
            #print(i)
            #del df[i]
        del df["Label"]
        del df['Unnamed: 0']
        df["Label"]=y
        last_name=f'./{outputfolder}/last_{str(WS)}_{j[5:]}'
        df.to_csv(last_name,index=None)
        print(WS,"-------------------------------------------------------------------------------")
